In [78]:
## Boiler plate code common to many notebooks.  See the TestFilesCommonCode.ipynb for details
from __future__ import print_function
%run TestFilesCommonCode.ipynb

SimpleITK Version: 0.9.1
Compiled: Sep 28 2015 10:07:41



In [79]:
import os
import glob
import sys

#\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/
#####################################################################################
#     Prepend the shell environment search paths
PROGRAM_PATHS = '/scratch/NAMICExternalProjects/release-20160523/bin'
#PROGRAM_PATHS = '/scratch/BS/release-BSR/bin'
PROGRAM_PATHS = PROGRAM_PATHS.split(':')
PROGRAM_PATHS.extend(os.environ['PATH'].split(':'))
os.environ['PATH'] = ':'.join(PROGRAM_PATHS)

CUSTOM_ENVIRONMENT=dict()

# Platform specific information
#     Prepend the python search paths
#PYTHON_AUX_PATHS = '/scratch/BS/BRAINSTools/AutoWorkup'
PYTHON_AUX_PATHS = '/scratch/SuperResolution/BRAINSSuperResolution/HCPWorkflows/'
PYTHON_AUX_PATHS = PYTHON_AUX_PATHS.split(':')
PYTHON_AUX_PATHS.extend(sys.path)
sys.path = PYTHON_AUX_PATHS

import SimpleITK as sitk
import nipype
from nipype.interfaces.base import CommandLine, CommandLineInputSpec, TraitedSpec, File, Directory
from nipype.interfaces.base import traits, isdefined, BaseInterface
from nipype.interfaces.utility import Merge, Split, Function, Rename, IdentityInterface
import nipype.interfaces.io as nio   # Data i/oS
import nipype.pipeline.engine as pe  # pypeline engine
from nipype.interfaces.freesurfer import ReconAll
from nipype.interfaces.ants import DenoiseImage
from nipype.interfaces.semtools import *

In [80]:
DWI_baseline = '/scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/Outputs_backup/SuperResolution/DWI_Baseline.nrrd'
DWI_sr_nn = '/scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/Outputs_backup/SuperResolution/DWI_SR_NN.nrrd'
DWI_sr_ifft = '/scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/Outputs_backup/SuperResolution/DWI_SR_IFFT.nrrd'
DWI_sr_tv = '/scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/Outputs_backup/SuperResolution/DWI_SR_TV.nrrd'
DWI_sr_wtv = '/scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/Outputs_backup/SuperResolution/DWI_SR_WTV.nrrd'
DWI_brainMask = '/scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/Outputs_backup/DWIBrainMask.nrrd'

In [81]:
output_dir = '/scratch/TESTS/IpythonNotebook/20160615_HCPWF/3_DistImagesWF/distImages_test'

In [84]:
def ComputeDistanceImages(DWI_baseline,DWI_sr,DWI_brainMask):
    import os
    import numpy as np
    from itertools import product
    import time
    from scipy.linalg import logm
    from scipy.linalg import eigvalsh # eigvalsh(A,B) is joint eigenvalues of A and B
    import SimpleITK as sitk
    from ReadWriteNrrdDWI import ReadNAMICDWIFromNrrd, WriteNAMICDWIToNrrd
    #########
    ### distance functions ###
    # Frobenius distance
    def distance_euclid(A, B):
        return np.linalg.norm(A - B, ord='fro')
    # Log-Euclidian distance
    def distance_logeuclid(A, B):
        return distance_euclid(logm(A),logm(B))
    # Reimannian distance
    def distance_reimann(A, B):
        return np.sqrt((np.log(eigvalsh(A,B))**2).sum())
    # Kullback-Leibler distance
    def distance_kullback(A, B):
        dim = A.shape[0]
        kl = np.sqrt( np.trace( np.dot(np.linalg.inv(A),B)+np.dot(np.linalg.inv(B),A) ) - 2*dim )
        return 0.5*kl
    ######### 
    assert os.path.exists(DWI_baseline), "File not found: %s" % DWI_baseline
    assert os.path.exists(DWI_sr), "File not found: %s" % DWI_sr
    assert os.path.exists(DWI_brainMask), "File not found: %s" % DWI_brainMask
    mask = sitk.ReadImage(DWI_brainMask)
    mask_data = sitk.GetArrayFromImage(mask)
    # read DWI nrrd files
    print('read nrrd files')
    data_base,nrrd_header_base,bvecs_base,bvals_base,grad_idx_base = ReadNAMICDWIFromNrrd(DWI_baseline)
    data_sr,nrrd_header_sr,bvecs_sr,bvals_sr,grad_idx_sr = ReadNAMICDWIFromNrrd(DWI_sr)
    # compute tensorfits
    print('compute tensorfits')
    from dipy.core.gradients import gradient_table
    from dipy.reconst.dti import TensorModel
    # base tensorfit
    gtab_base = gradient_table(bvals_base, bvecs_base)
    ten_base = TensorModel(gtab_base)
    tenfit_base = ten_base.fit(data_base)
    # super-res reconstructed tensorfit
    gtab_sr = gradient_table(bvals_sr, bvecs_sr)
    ten_sr = TensorModel(gtab_sr)
    tenfit_sr = ten_base.fit(data_sr)
    ## create RIS error arrays
    fa_distance_arr = abs(tenfit_base.fa-tenfit_sr.fa)
    md_distance_arr = abs(tenfit_base.md-tenfit_sr.md)
    rd_distance_arr = abs(tenfit_base.rd-tenfit_sr.rd)
    ad_distance_arr = abs(tenfit_base.ad-tenfit_sr.ad)
    ## mask out the background
    fa_distance_arr = np.transpose(fa_distance_arr,(2, 1, 0))
    md_distance_arr = np.transpose(md_distance_arr,(2, 1, 0))
    rd_distance_arr = np.transpose(rd_distance_arr,(2, 1, 0))
    ad_distance_arr = np.transpose(ad_distance_arr,(2, 1, 0))
    #
    fa_distance_arr = fa_distance_arr*mask_data
    md_distance_arr = md_distance_arr*mask_data
    rd_distance_arr = rd_distance_arr*mask_data
    ad_distance_arr = ad_distance_arr*mask_data
    ## clip outliers by computing 'p' percentiles
    p = 99.7
    np.clip(fa_distance_arr, fa_distance_arr.min(), np.percentile(fa_distance_arr,p), fa_distance_arr)
    np.clip(md_distance_arr, md_distance_arr.min(), np.percentile(md_distance_arr,p), md_distance_arr)
    np.clip(rd_distance_arr, rd_distance_arr.min(), np.percentile(rd_distance_arr,p), rd_distance_arr)
    np.clip(ad_distance_arr, ad_distance_arr.min(), np.percentile(ad_distance_arr,p), ad_distance_arr)
    ## create RIS distance images
    fa_distance_image = sitk.GetImageFromArray(fa_distance_arr)
    fa_distance_image.CopyInformation(mask)
    #
    md_distance_image = sitk.GetImageFromArray(md_distance_arr)
    md_distance_image.CopyInformation(mask)
    #
    rd_distance_image = sitk.GetImageFromArray(rd_distance_arr)
    rd_distance_image.CopyInformation(mask)
    #
    ad_distance_image = sitk.GetImageFromArray(ad_distance_arr)
    ad_distance_image.CopyInformation(mask)
    #
    size = [data_base.shape[0],data_base.shape[1],data_base.shape[2]] #data_base.shape[4] is gradient components
    frobenius_distance_arr = np.empty(size, dtype=float)
    logeuclid_distance_arr = np.empty(size, dtype=float)
    reimann_distance_arr = np.empty(size, dtype=float)
    kullback_distance_arr = np.empty(size, dtype=float)
    #
    tensors_base = tenfit_base.quadratic_form
    tensors_sr = tenfit_sr.quadratic_form
    # for loop to find other distance images
    print('run for loop')
    tstart = time.time()
    for i, idx in enumerate(product(xrange(size[0]),xrange(size[1]),xrange(size[2]))):
        frobenius_distance_arr[idx] = distance_euclid(tensors_base[idx], tensors_sr[idx])
        ## HACK: log euclidian metric generates results the same as Reimannian metric,
        ## but it takes way longer (about 70 times longer!!!)
        #logeuclid_distance_arr[idx] = distance_logeuclid(tensors_base[idx], tensors_sr[idx])
        reimann_distance_arr[idx] = distance_reimann(tensors_base[idx], tensors_sr[idx])
        kullback_distance_arr[idx] = distance_kullback(tensors_base[idx], tensors_sr[idx])
    elapsed = time.time() - tstart
    print('for loop took: ', elapsed)
    ## HACK: set values of logeuclid_distance_arr to zero
    logeuclid_distance_arr.fill(0)
    # mask out the background
    frobenius_distance_arr = np.transpose(frobenius_distance_arr,(2, 1, 0))
    logeuclid_distance_arr = np.transpose(logeuclid_distance_arr,(2, 1, 0))
    reimann_distance_arr = np.transpose(reimann_distance_arr,(2, 1, 0))
    kullback_distance_arr = np.transpose(kullback_distance_arr,(2, 1, 0))
    #
    frobenius_distance_arr = frobenius_distance_arr*mask_data
    logeuclid_distance_arr = logeuclid_distance_arr*mask_data
    reimann_distance_arr = reimann_distance_arr*mask_data
    kullback_distance_arr = kullback_distance_arr*mask_data
    ## clip outliers by computing 'p' percentiles
    p = 99.7
    np.clip(frobenius_distance_arr, frobenius_distance_arr.min(), np.percentile(frobenius_distance_arr,p), frobenius_distance_arr)
    p = 99.6
    np.clip(logeuclid_distance_arr, logeuclid_distance_arr.min(), np.percentile(logeuclid_distance_arr,p), logeuclid_distance_arr)
    np.clip(reimann_distance_arr, reimann_distance_arr.min(), np.percentile(reimann_distance_arr,p), reimann_distance_arr)
    np.clip(kullback_distance_arr, kullback_distance_arr.min(), np.percentile(kullback_distance_arr,p), kullback_distance_arr)
    ## create RIS distance images
    frobenius_distance_image = sitk.GetImageFromArray(frobenius_distance_arr)
    frobenius_distance_image.CopyInformation(mask)
    #
    logeuclid_distance_image = sitk.GetImageFromArray(logeuclid_distance_arr)
    logeuclid_distance_image.CopyInformation(mask)
    #
    reimann_distance_image = sitk.GetImageFromArray(reimann_distance_arr)
    reimann_distance_image.CopyInformation(mask)
    #
    kullback_distance_image = sitk.GetImageFromArray(kullback_distance_arr)
    kullback_distance_image.CopyInformation(mask)
    ## create proper SR prefix for output file name
    sr_file_name = os.path.basename(DWI_sr)
    sr_file_name_base = os.path.splitext(sr_file_name)[0]
    sr_name = sr_file_name_base.split('_',2)[2]
    #
    fa_distance_image_fn = os.path.join(output_dir,sr_name + '_FA_distance.nrrd')
    md_distance_image_fn = os.path.join(output_dir,sr_name + '_MD_distance.nrrd')
    rd_distance_image_fn = os.path.join(output_dir,sr_name + '_RD_distance.nrrd')
    ad_distance_image_fn = os.path.join(output_dir,sr_name + '_AD_distance.nrrd')
    frobenius_distance_image_fn = os.path.join(output_dir,sr_name + '_Frobenius_distance.nrrd')
    logeuclid_distance_image_fn = os.path.join(output_dir,sr_name + '_Logeuclid_distance.nrrd')
    reimann_distance_image_fn = os.path.join(output_dir,sr_name + '_Reimann_distance.nrrd')
    kullback_distance_image_fn = os.path.join(output_dir,sr_name + '_Kullback_distance.nrrd')
    ## Write out the distance images
    sitk.WriteImage(fa_distance_image,fa_distance_image_fn)
    sitk.WriteImage(md_distance_image,md_distance_image_fn)
    sitk.WriteImage(rd_distance_image,rd_distance_image_fn)
    sitk.WriteImage(ad_distance_image,ad_distance_image_fn)
    sitk.WriteImage(frobenius_distance_image,frobenius_distance_image_fn)
    sitk.WriteImage(logeuclid_distance_image,logeuclid_distance_image_fn)
    sitk.WriteImage(reimann_distance_image,reimann_distance_image_fn)
    sitk.WriteImage(kullback_distance_image,kullback_distance_image_fn)
    #
    return [fa_distance_image_fn,
            md_distance_image_fn,
            rd_distance_image_fn,
            ad_distance_image_fn,
            frobenius_distance_image_fn,
            logeuclid_distance_image_fn,
            reimann_distance_image_fn,
            kullback_distance_image_fn]

In [85]:
import time
tfuncstart = time.time()

[fa,md,rd,ad,fro,log,rei,kull] = ComputeDistanceImages(DWI_baseline,DWI_sr_nn,DWI_brainMask)

elapsedfunc = time.time() - tfuncstart
print('whole function took: ', elapsedfunc)

print(fa)

read nrrd files
compute tensorfits
run for loop
for loop took:  784.437544823
whole function took:  1694.38914704
/scratch/TESTS/IpythonNotebook/20160615_HCPWF/3_DistImagesWF/distImages_test/NN_FA_distance.nrrd


In [ ]:
# find best percentile for Reimann/Kullback dist images

In [73]:
rei_fn = '/scratch/TESTS/IpythonNotebook/20160615_HCPWF/3_DistImagesWF/distImages_test/NN_Reimann_distance.nrrd'
kull_fn = '/scratch/TESTS/IpythonNotebook/20160615_HCPWF/3_DistImagesWF/distImages_test/NN_Kullback_distance.nrrd'
fro_fn = '/scratch/TESTS/IpythonNotebook/20160615_HCPWF/3_DistImagesWF/distImages_test/NN_Frobenius_distance.nrrd'

fro = sitk.ReadImage(fro_fn)
rei = sitk.ReadImage(rei_fn)
kull = sitk.ReadImage(kull_fn)

frobenius_distance_arr = sitk.GetArrayFromImage(fro)
reimann_distance_arr = sitk.GetArrayFromImage(rei)
kullback_distance_arr = sitk.GetArrayFromImage(kull)

In [63]:
print(reimann_distance_arr.min())
print(reimann_distance_arr.max())
print(reimann_distance_arr.mean())
#print(np.median(reimann_distance_arr))
print(np.percentile(reimann_distance_arr,99.6))

0.0
7.40247695441
0.0880115014564
2.60121282239


In [72]:
print(kullback_distance_arr.min())
print(kullback_distance_arr.max())
print(kullback_distance_arr.mean())
#print(np.median(kullback_distance_arr))
print(np.percentile(kullback_distance_arr,99.6))

0.0
8.10893536737
0.0584255590451
1.46756204569


In [77]:
print(frobenius_distance_arr.min())
print(frobenius_distance_arr.max())
print(frobenius_distance_arr.mean())
#print(np.median(frobenius_distance_arr))
print(np.percentile(frobenius_distance_arr,99.6))

0.0
0.00105114968574
4.60893588125e-05
0.000918160157245


In [20]:
# # Do sum tests
# fro_fn = '/scratch/TESTS/IpythonNotebook/20160615_HCPWF/3_DistImagesWF/distImages_test_backup/NN_Frobenius_distance.nrrd'
# logeuc_fn = '/scratch/TESTS/IpythonNotebook/20160615_HCPWF/3_DistImagesWF/distImages_test_backup/NN_Logeuclid_distance.nrrd'
# rei_fn = '/scratch/TESTS/IpythonNotebook/20160615_HCPWF/3_DistImagesWF/distImages_test_backup/NN_Reimann_distance.nrrd'
# kull_fn = '/scratch/TESTS/IpythonNotebook/20160615_HCPWF/3_DistImagesWF/distImages_test_backup/NN_Kullback_distance.nrrd'
# fa = sitk.ReadImage(fro_fn)
# md = sitk.ReadImage(logeuc_fn)
# rd = sitk.ReadImage(rei_fn)
# ad = sitk.ReadImage(kull_fn)
# statFilter = sitk.StatisticsImageFilter()
# print('-----Fro-------')
# statFilter.Execute(fa)
# print(statFilter.GetMaximum())
# print(statFilter.GetMinimum())
# print(statFilter.GetMean())
# print('-----LogEuc-------')
# statFilter.Execute(md)
# print(statFilter.GetMaximum())
# print(statFilter.GetMinimum())
# print(statFilter.GetMean())
# print('-----Reimann-------')
# statFilter.Execute(rd)
# print(statFilter.GetMaximum())
# print(statFilter.GetMinimum())
# print(statFilter.GetMean())
# print('-----Kullback-------')
# statFilter.Execute(ad)
# print(statFilter.GetMaximum())
# print(statFilter.GetMinimum())
# print(statFilter.GetMean())

# ## NN_base images (using outlier of %95)
# print('===================================')
# fro_fn = '/scratch/TESTS/IpythonNotebook/20160615_HCPWF/3_DistImagesWF/distImages_test/NN_Frobenius_distance.nrrd'
# logeuc_fn = '/scratch/TESTS/IpythonNotebook/20160615_HCPWF/3_DistImagesWF/distImages_test/NN_Logeuclid_distance.nrrd'
# rei_fn = '/scratch/TESTS/IpythonNotebook/20160615_HCPWF/3_DistImagesWF/distImages_test/NN_Reimann_distance.nrrd'
# kull_fn = '/scratch/TESTS/IpythonNotebook/20160615_HCPWF/3_DistImagesWF/distImages_test/NN_Kullback_distance.nrrd'
# fa = sitk.ReadImage(fro_fn)
# md = sitk.ReadImage(logeuc_fn)
# rd = sitk.ReadImage(rei_fn)
# ad = sitk.ReadImage(kull_fn)
# print('-----Fro-------')
# statFilter.Execute(fa)
# print(statFilter.GetMaximum())
# print(statFilter.GetMinimum())
# print(statFilter.GetMean())
# print('-----LogEuc-------')
# statFilter.Execute(md)
# print(statFilter.GetMaximum())
# print(statFilter.GetMinimum())
# print(statFilter.GetMean())
# print('-----Reimann-------')
# statFilter.Execute(rd)
# print(statFilter.GetMaximum())
# print(statFilter.GetMinimum())
# print(statFilter.GetMean())
# print('-----Kullback-------')
# statFilter.Execute(ad)
# print(statFilter.GetMaximum())
# print(statFilter.GetMinimum())
# print(statFilter.GetMean())

-----Fro-------
0.00105114968574
0.0
4.60893588125e-05
-----LogEuc-------
7.23208259506
0.0
0.0873063250486
-----Reimann-------
7.40247695441
0.0
0.0880115014564
-----Kullback-------
8.10893536737
0.0
0.0584255590451
-----Fro-------
0.00105114968574
0.0
4.60893588125e-05
-----LogEuc-------
0.0
0.0
0.0
-----Reimann-------
7.40247695441
0.0
0.0880115014564
-----Kullback-------
8.10893536737
0.0
0.0584255590451
